In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0


# QAOA + Quantum Minimum Finding for LABS

The **Low Autocorrelation Binary Sequences (LABS)** problem is a binary optimization problem with applications in radar and communications. This notebook implements the approach from [*Evidence of scaling advantage for the quantum approximate optimization algorithm on a classically intractable problem*](https://www.science.org/doi/10.1126/sciadv.adm6761) (Science, 2024): **QAOA for LABS** combined with **Quantum Minimum Finding (QMF)** as an amplitude amplifier.

Key result from the paper: QAOA alone has time-to-solution (TTS) scaling ~1.46^N; **QAOA+QMF** achieves the best empirical scaling ~**1.21^N**, better than the best classical heuristic (Memetic Tabu, ~1.34^N).

**In this notebook you will:**
1. Understand the LABS problem and its cost Hamiltonian (Eq. 4).
2. Implement QAOA for LABS in CUDA-Q (phase operator + mixer, fixed parameters).
3. Use QMF conceptually as an amplitude amplifier on the QAOA state and simulate its effect (repeated sampling + keep best).

**Prerequisites:** Basic quantum computing (gates, measurement). CUDA-Q: `cudaq.sample()`, `@cudaq.kernel`, `rx`, `rz`, `x`, `h`, `x.ctrl()`.


In [ ]:
import cudaq
import numpy as np
from math import floor


## The LABS problem and applications

The **Low Autocorrelation Binary Sequences (LABS)** problem is fundamental to many applications, but originated with applications to radar. 

Consider a radar that monitors airport traffic.  The radar signal sent to detect incoming planes must have as much range as possible to ensure safe approaches are planned well in advance.  The range of a radar signal can be increased by sending a longer pulse.  However, in order to differentiate between multiple objects, pulses need to be short to provide high resolution. So, how do you handle situations where you need both?

One solution is a technique called pulse compression.  The idea is to send a long signal, but vary the phase at regular intervals such that the resolution is increased. Generally, the initial signal will encode a binary sequence of phase shifts, where each interval corresponds to a signal with a 0 or 180 degree phase shift. 

The tricky part is selecting an optimal encoding sequence.  When the signal returns, it is fed into a matched filter with the hope that a singular sharp peak will appear, indicating clear detection.  The autocorrelation of the original signal, or how similar the signal is to itself,  determines if a single peak or a messier signal with sidelobes will be detected. A signal should have high autocorrelation when overlayed on top of itself, but low autocorrelation when shifted with a lag. 

Consider the image below.  The signal on the left has a crisp single peak while the single on the right produces many undesirable sidelobes which can inhibit clear detection.  

<img src="images/quantum_enhanced_optimization_LABS/radar.png" width="800">


So, how do you select a good signal?   This is where LABS comes in, defining these questions as a binary optimization problem. Given a binary sequence of length $N$, $(s_1 \cdots s_N) \in {\pm 1}^N$, the goal is to minimize the following objective function.

$$ E(s) = \sum_{k=1}^{N-1} C_k^2 $$

Where $C_k$ is defined as. 

 $$C_k= \sum_{i=1}^{N-k} s_is_{i+k}$$


So, each $C_k$ computes how similar the original signal is to the shifted one for each offset value $k$.  To explore this more, try the interactive widget linked [here](https://nvidia.github.io/cuda-q-academic/interactive_widgets/labs_visualization.html).  See if you can select a very good and very poor sequence and see the difference for the case of $N=7$.

## Classical context

The best classical heuristic for LABS is **Memetic Tabu search (MTS)**, with TTS scaling $O(1.34^N)$. The Science paper shows that **QAOA+QMF** achieves better empirical scaling ($\sim 1.21^N$) by using constant-depth QAOA to prepare a state with good overlap to the optimum, then applying quantum minimum finding (amplitude amplification with threshold search) as a subroutine. We implement QAOA for LABS and simulate the QMF amplification effect below.


In [ ]:
def get_interactions(N):
    """LABS Hamiltonian (Science Eq. 4): G2 = two-body z_i z_{i+2k}, G4 = four-body z_i z_{i+t} z_{i+k} z_{i+k+t}."""
    G2 = []
    G4 = []
    for i in range(N - 2):
        for k in range(1, int(floor((N - i) / 2)) + 1):
            j = i + 2 * k
            if j < N:
                G2.append([i, j])
    for i in range(N - 3):
        for t in range(1, int(floor((N - i - 1) / 2)) + 1):
            for k in range(t + 1, N - i - t):
                if i + k + t < N:
                    G4.append([i, i + t, i + k, i + k + t])
    return G2, G4

def compute_Ck(s, k):
    """C_k = sum_i s_i s_{i+k} (s is ±1 sequence)."""
    N = len(s)
    return sum(s[i] * s[i + k] for i in range(N - k))

def compute_energy(s):
    """Sidelobe energy E(s) = sum_{k=1}^{N-1} C_k^2."""
    N = len(s)
    return sum(compute_Ck(s, k) ** 2 for k in range(1, N))

def compute_merit_factor(s, energy=None):
    """Merit factor F(s) = N^2 / (2*E(s))."""
    N = len(s)
    if energy is None:
        energy = compute_energy(s)
    return (N * N) / (2 * energy) if energy > 0 else float('inf')

def bitstring_to_sequence(bitstring):
    """'0' -> +1, '1' -> -1."""
    return np.array([1 if b == '0' else -1 for b in bitstring])

def sequence_to_bitstring(s):
    """+1 -> '0', -1 -> '1'."""
    return ''.join('0' if x == 1 else '1' for x in s)

# Quick check
G2, G4 = get_interactions(5)
print("N=5: G2 terms =", len(G2), ", G4 terms =", len(G4))


## QAOA for LABS

QAOA prepares the state $|\beta,\gamma\rangle = \prod_{l=1}^p e^{-i\beta_l \sum_j X_j} e^{-i\gamma_l H_C} |+\rangle^{\otimes N}$, where $H_C$ is the LABS cost Hamiltonian (Eq. 4 in the Science paper): two-body $z_i z_{i+2k}$ and four-body $z_i z_{i+t} z_{i+k} z_{i+k+t}$ terms. We use fixed parameters: $\beta_l$ and $\gamma_l$ with $\gamma$ rescaled by $1/N$ for transferability.


In [ ]:
@cudaq.kernel
def rzz_gate(q0: cudaq.qubit, q1: cudaq.qubit, theta: float):
    """exp(-i theta/2 Z⊗Z) via CNOT-RZ-CNOT."""
    x.ctrl(q0, q1)
    rz(theta, q1)
    x.ctrl(q0, q1)

@cudaq.kernel
def qaoa_kernel(N: int, G2: list[list[int]], G4: list[list[int]],
               num_layers: int, betas: list[float], gammas: list[float]):
    """QAOA: |+>^N then for each layer: phase exp(-i gamma H_C), mixer exp(-i beta sum X_j)."""
    qubits = cudaq.qvector(N)
    h(qubits)
    for layer in range(num_layers):
        gamma = gammas[layer]
        beta = betas[layer]
        for term in G2:
            rzz_gate(qubits[term[0]], qubits[term[1]], 2.0 * gamma)
        for term in G4:
            i0, i1, i2, i3 = term[0], term[1], term[2], term[3]
            x.ctrl(qubits[i0], qubits[i1])
            x.ctrl(qubits[i1], qubits[i2])
            x.ctrl(qubits[i2], qubits[i3])
            rz(4.0 * gamma, qubits[i3])
            x.ctrl(qubits[i2], qubits[i3])
            x.ctrl(qubits[i1], qubits[i2])
            x.ctrl(qubits[i0], qubits[i1])
        for j in range(N):
            rx(2.0 * beta, qubits[j])

def get_fixed_parameters(p, N):
    """Fixed schedule: beta linear ramp, gamma rescaled by 1/N."""
    betas = [0.5 * np.pi * (l + 1) / (p + 1) for l in range(p)]
    gammas = [0.25 * np.pi * (l + 1) / ((p + 1) * N) for l in range(p)]
    return betas, gammas

def run_qaoa(N, p=2, shots=500):
    """Run QAOA and return samples (bitstring -> count), best bitstring, best energy, expected merit."""
    G2, G4 = get_interactions(N)
    betas, gammas = get_fixed_parameters(p, N)
    result = cudaq.sample(qaoa_kernel, N, G2, G4, p, betas, gammas, shots_count=shots)
    samples = dict(result)
    best_bs, best_e, best_m = None, float('inf'), 0.0
    total = sum(samples.values())
    expected_merit = 0.0
    for bs, count in samples.items():
        s = bitstring_to_sequence(bs)
        e = compute_energy(s)
        m = compute_merit_factor(s, e)
        expected_merit += (count / total) * m
        if e < best_e:
            best_e, best_bs, best_m = e, bs, m
    return samples, best_bs, best_e, best_m, expected_merit

N, p, shots = 10, 2, 500
samples, best_bs, best_e, best_m, exp_m = run_qaoa(N, p, shots)
print(f"QAOA (N={N}, p={p}, shots={shots}): best energy={best_e}, best merit={best_m:.4f}, expected merit={exp_m:.4f}")
print("Top 3 bitstrings:", sorted(samples.items(), key=lambda x: -x[1])[:3])


## QMF as amplitude amplifier

**Quantum Minimum Finding (QMF)** (Dürr–Høyer) reduces optimization to feasibility: set a cost threshold, use an oracle that marks states with energy below the threshold, and run amplitude amplification (Grover-style). The classical loop updates the threshold (e.g. binary search). The Science paper uses a *generalized* QMF that accepts an **arbitrary initial state**: the QAOA state (instead of uniform superposition) has much better overlap with the optimum, so QAOA+QMF achieves ~1.21^N TTS.

Here we **simulate the effect** of QMF amplification: we run QAOA sampling multiple times and keep the best solution found (iterative minimum-finding in post-processing). This illustrates how amplitude amplification would improve over single-shot QAOA.


In [ ]:
def run_qaoa_plus_qmf_simulated(N, p=2, shots_per_round=200, num_rounds=5):
    """Simulate QAOA+QMF: multiple QAOA sampling rounds, keep best solution (min-finding)."""
    best_bs, best_e, best_m = None, float('inf'), 0.0
    all_best_energies = []
    for r in range(num_rounds):
        samples, bs, e, m, _ = run_qaoa(N, p, shots_per_round)
        all_best_energies.append(e)
        if e < best_e:
            best_e, best_bs, best_m = e, bs, m
    return best_bs, best_e, best_m, all_best_energies

best_bs, best_e, best_m, energies = run_qaoa_plus_qmf_simulated(N=10, p=2, shots_per_round=200, num_rounds=5)
print("QAOA+QMF (simulated): best energy =", best_e, ", best merit =", f"{best_m:.4f}")
print("Energies per round:", energies)


## Summary

- **QAOA for LABS** uses the cost Hamiltonian $H_C$ (Eq. 4) and fixed-parameter schedules; it produces a biased distribution over bitstrings with better overlap to low-energy solutions than random guessing.
- **QMF as amplitude amplifier** uses the QAOA state as the initial state and amplifies the probability of measuring solutions below a threshold; together **QAOA+QMF** gives the best empirical TTS scaling (~1.21^N) for LABS (Science, 2024).
- In this notebook we implemented QAOA in CUDA-Q and simulated the QMF effect by repeated QAOA sampling and keeping the best solution.
